In [1]:
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import roc_auc_score, confusion_matrix

In [2]:
tel = pd.read_csv('data/telecom_data.csv').drop(['customerID', 'index'], axis=1)
tel.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,...,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,tenure_group
0,Male,No,No,No,34,Yes,No,DSL,Yes,No,...,No,No,No,One year,No,Mailed check,56.95,1889.50,No,Tenure_24-48
1,Male,No,No,No,2,Yes,No,DSL,Yes,Yes,...,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,Tenure_0-12
2,Male,No,No,No,45,No,No phone service,DSL,Yes,No,...,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No,Tenure_24-48
3,Female,No,No,No,2,Yes,No,Fiber optic,No,No,...,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,Tenure_0-12
4,Female,No,No,No,8,Yes,Yes,Fiber optic,No,No,...,No,Yes,Yes,Month-to-month,Yes,Electronic check,99.65,820.50,Yes,Tenure_0-12


## Data preprocessing (5).  
This time we will skip all the exploration steps and only do some simple features preprocessing:
- Generate `tenure_group` attribute: discretize `tenure` into 6 groups: “0-12”, “12-24”, “24-36”, “36-48”, “48-60”, “60+” (all are left closed intervals: \[0, 12), \[12, 24), \[24, 36)...). What are the sizes of these groups? Tenure refers to the number of months that a customer has subscribed for. Do not drop the `tenure` column.  
- Preprocess categorical columns with only 2 unique values (“binary” columns): replace one unique value with 0 and another with 1 (label encoding). How many such columns do you have?
E.g. for the `gender` attribute you may replace Female with 1 and Male with 0 or vice versa. 
- Preprocess categorical columns with more then 2 unique values using dummy encoding (=one-hot encoding). How many such columns (before dummy encoding) do you have? 
Drop customerID attribute.

In [3]:
tel['tenure_group'] = 1
tel.loc[(12 <= tel.tenure) & (tel.tenure < 24), 'tenure_group'] = 2
tel.loc[(24 <= tel.tenure) & (tel.tenure < 36), 'tenure_group'] = 3
tel.loc[(36 <= tel.tenure) & (tel.tenure < 48), 'tenure_group'] = 4
tel.loc[(48 <= tel.tenure) & (tel.tenure < 60), 'tenure_group'] = 5
tel.loc[60 <= tel.tenure, 'tenure_group'] = 6

tel.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,...,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,tenure_group
0,Male,No,No,No,34,Yes,No,DSL,Yes,No,...,No,No,No,One year,No,Mailed check,56.95,1889.50,No,3
1,Male,No,No,No,2,Yes,No,DSL,Yes,Yes,...,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,1
2,Male,No,No,No,45,No,No phone service,DSL,Yes,No,...,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No,4
3,Female,No,No,No,2,Yes,No,Fiber optic,No,No,...,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,1
4,Female,No,No,No,8,Yes,Yes,Fiber optic,No,No,...,No,Yes,Yes,Month-to-month,Yes,Electronic check,99.65,820.50,Yes,1


In [4]:
tel.groupby(['tenure_group']).tenure.count().reset_index(name='count')

,tenure_group,count
0,1,1573
1,2,844
2,3,719
3,4,670
4,5,775
5,6,1462


In [5]:
cat_cols = tel.select_dtypes(include=['object']).columns
bin_cat_cols = [cat_col for cat_col in cat_cols if len(tel[cat_col].unique()) == 2]
nonbin_cat_cols = [cat_col for cat_col in cat_cols if len(tel[cat_col].unique()) != 2]

print(f'binary categorical columns: {len(bin_cat_cols)}')
print(f'nonbinary categorical columns: {len(nonbin_cat_cols)}')

binary categorical columns: 13
nonbinary categorical columns: 4


In [6]:
le = LabelEncoder()

for col in bin_cat_cols:
    tel[col] = le.fit_transform(tel[col])
for col in nonbin_cat_cols:
    tel = pd.get_dummies(tel, prefix=[col], columns=[col], drop_first=True)

tel.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,...,tenure_group,MultipleLines_No phone service,MultipleLines_Yes,InternetService_Fiber optic,InternetService_No,Contract_One year,Contract_Two year,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,1,0,0,0,34,1,1,0,1,0,...,3,0,0,0,0,1,0,0,0,1
1,1,0,0,0,2,1,1,1,0,0,...,1,0,0,0,0,0,0,0,0,1
2,1,0,0,0,45,0,1,0,1,1,...,4,1,0,0,0,1,0,0,0,0
3,0,0,0,0,2,1,0,0,0,0,...,1,0,0,1,0,0,0,0,1,0
4,0,0,0,0,8,1,0,0,1,0,...,1,0,1,1,0,0,0,0,1,0


## Build a churn model (5).
- Build 2 classification models to predict customers churn:
    - Logistic Regression. What is the ROC AUC of this model?
    - Random Forest. What is the ROC AUC of this model?

*In this task I suggest you deviate from the train-test strategy and use a k-fold approach to train and predict the whole dataset: train on ⅘ of the data, predict on ⅕ , repeat this 5 times, thus you will get predictions for the whole dataset. Recall that in this case you will actually have 5 trained classifiers not 1, but for our purposes this is ok.*
After this section you must have predictions for all customers from the dataset (~6k) obtained using cross validation.

Comments:
For the section III you may use any classification model you want (you are not restricted by these two models above).
You may want to use grid search to look for the best parameters of the model/s.

In [7]:
y = tel.Churn
X = tel.drop(['Churn'], axis=1)
THRESHOLD = 0.5

In [8]:
lr = LogisticRegression(max_iter=1000)
lr_proba = cross_val_predict(lr, X, y, method='predict_proba')
lr_pred = [int(item[1] >= THRESHOLD) for item in lr_proba]
roc_auc_score(y, lr_pred)

0.8533170384168807

In [9]:
rf = RandomForestClassifier()
rf_proba = cross_val_predict(rf, X, y, method='predict_proba')
rf_pred = [int(item[1] >= THRESHOLD) for item in rf_proba]
roc_auc_score(y, rf_pred)

0.8425283859957038

## Compare two discount strategies (25 + 10).
Assumptions:
- Every customer pays the same price p which is the average of `MonthlyCharges`.
- If we decide to provide a discount we provide it to all the customers who are predicted as Churn=Yes.
- When we compute gains, costs and losses we compute them for the short term.
Therefore all the computations from the seminar hold (except you need to recompute the coefficients).
- Strategy’s profit is the difference between gains, costs and losses: 
profit = gains - costs - losses
- Profit per customer is the total profit divided by the number of customers (if the person churns the person is not a customer anymore).

**Strategy A**: Provide a 20% discount with a 75% acceptance rate.  
**Strategy B**: Provide a 30% discount with a 90% acceptance rate.
In the seminar we had a 30% discount with 80% acceptance rate.

- Use the default threshold of 0.5 to compute the confusion matrix. Based on this confusion matrix report (5 points):
    - TP, FP, TN, FN
    - Losses if you do not apply any discount strategy.
    - Total gains from the discount strategy B.
    - Total costs of the discount strategy B.
    - Total losses of the discount strategy B.
    - Total profit of the discount strategy B.
    - Profit per customer pd(using strategy B). 
- Use 9 different thresholds: 0.1, 0.2, … 0.9. Answer to the following questions (10 points):
    - What is the threshold with the highest accuracy, using strategy B?
    - What is the threshold with the highest profit, using strategy B? What is the highest profit, using strategy B?
    - What is the threshold with the highest profit per customer, using strategy B ?  What is the highest profit per customer, using strategy B?
    - What is the ratio of profit per customer (obtained on the previous step) and p?
    - Which strategy yields the highest profit (A or B)? What are the TP, FP, TN, FN in this case? What is the highest profit in that case?
    - Which strategy yields the highest profit per customer (A or B)? What are the TP, FP, TN, FN in this case? What is the highest profit per customer in that case?
- Prepare a report (10 points). Your report must summarize your results. Reports with simple copy paste of the results will be graded with 0 points! Some example questions (you are not limited or restricted to them): 
    - Do thresholds for the highest profit and highest profit per customer coincide or not? Why? 
    - Which you decide to choose? Under what circumstances (how many clients will you lose in both situations; what should be the decision criteria)?
    - How hard does your profit per customer decrease for customers for whom you provide a discount, compared to customers for whom you do not provide a discount? Compare this number with the discount.

In [10]:
p = tel.MonthlyCharges.mean()
p

62.85776104583816

In [11]:
def metrics(cm, acceptance, discount):
    tn, fp, fn, tp = cm.ravel()
    accuracy = (tp + tn) / (tp + fp + fn + tn)
    
    losses_do_nothing = (tp + fn) * p
    total_gains = tp * acceptance * p
    total_costs = (tp + fp) * discount * p
    total_losses = fn * p + tp * (1 - acceptance) * (1 - discount) * p
    total_profit = total_gains - total_costs - total_losses
    profit_per_customer = total_profit / (tp * acceptance + fp + tn)
    
    return pd.DataFrame({'tn': [tn],
                         'fp': [fp],
                         'fn': [fn],
                         'tp': [tp],
                         'accuracy': [accuracy],
                         'losses_do_nothing': [losses_do_nothing],
                         'total_gains': [total_gains],
                         'total_costs': [total_costs],
                         'total_losses': [total_losses],
                         'total_profit': [total_profit],
                         'profit_per_customer': [profit_per_customer],
                         'PPC/p': [profit_per_customer / p]})

In [12]:
cm = confusion_matrix(y, lr_pred)
tn, fp, fn, tp = cm.ravel()
tn, fp, fn, tp

(3950, 224, 448, 1421)

In [13]:
metrics(cm, 0.9, 0.3)

,tn,fp,fn,tp,accuracy,losses_do_nothing,total_gains,total_costs,total_losses,total_profit,profit_per_customer,PPC/p
0,3950,224,448,1421,0.888797,117481.155395,80388.790602,31020.305076,34412.73844,14955.747086,2.742714,0.043634


In [14]:
strategy_A = pd.DataFrame(columns=['threshold', 'tn', 'fp', 'fn', 'tp', 'accuracy', 'losses_do_nothing', 'total_gains', 'total_costs', 'total_losses', 'total_profit', 'profit_per_customer'])
for i in range(1, 10):
    threshold = .1 * i
    
    lr = LogisticRegression(max_iter=1000)
    lr_proba = cross_val_predict(lr, X, y, method='predict_proba')
    lr_pred = [int(item[1] >= threshold) for item in lr_proba]
    
    cm = confusion_matrix(y, lr_pred)
    
    m = metrics(cm, 0.75, 0.2)
    m['threshold'] = threshold 
    strategy_A = strategy_A.append(m, sort=False, ignore_index=True)

strategy_A

,threshold,tn,fp,fn,tp,accuracy,losses_do_nothing,total_gains,total_costs,total_losses,total_profit,profit_per_customer,PPC/p
0,0.1,2653,1521,121,1748,0.728281,117481.155395,82406.524731,41096.404172,29580.862348,11729.258211,2.138424,0.034020
1,0.2,3164,1010,206,1663,0.798775,117481.155395,78399.342464,33603.759055,33855.190099,10940.393310,2.018057,0.032105
2,0.3,3514,660,294,1575,0.842131,117481.155395,74250.730235,28097.419187,38280.376477,7872.934571,1.470134,0.023388
3,0.4,3762,412,379,1490,0.869105,117481.155395,70243.547969,23911.092302,42554.704228,3777.751439,0.713928,0.011358
4,0.5,3950,224,448,1421,0.888797,117481.155395,66990.658835,20680.203384,46024.452638,286.002813,0.054583,0.000868
5,0.6,4085,89,542,1327,0.895582,117481.155395,62559.186681,17801.317928,50751.356268,-5993.487516,-1.159450,-0.018446
6,0.7,4142,32,639,1230,0.888962,117481.155395,57986.284565,15865.298888,55629.118526,-13508.132849,-2.650472,-0.042166
7,0.8,4171,3,767,1102,0.872580,117481.155395,51951.939504,13891.565191,62065.753257,-24005.378943,-4.800596,-0.076372
8,0.9,4174,0,1016,853,0.831872,117481.155395,40213.252629,10723.534034,74587.019257,-45097.300662,-9.368434,-0.149042


In [15]:
strategy_B = pd.DataFrame(columns=['threshold', 'tn', 'fp', 'fn', 'tp', 'accuracy', 'losses_do_nothing', 'total_gains', 'total_costs', 'total_losses', 'total_profit', 'profit_per_customer'])
for i in range(1, 10):
    threshold = .1 * i
    
    lr = LogisticRegression(max_iter=1000)
    lr_proba = cross_val_predict(lr, X, y, method='predict_proba')
    lr_pred = [int(item[1] >= threshold) for item in lr_proba]
    
    cm = confusion_matrix(y, lr_pred)
    
    m = metrics(cm, 0.9, 0.3)
    m['threshold'] = threshold 
    strategy_B = strategy_B.append(m, sort=False, ignore_index=True)

strategy_B

,threshold,tn,fp,fn,tp,accuracy,losses_do_nothing,total_gains,total_costs,total_losses,total_profit,profit_per_customer,PPC/p
0,0.1,2653,1521,121,1748,0.728281,117481.155395,98887.829677,61644.606258,15297.064728,21946.158692,3.818583,0.060750
1,0.2,3164,1010,206,1663,0.798775,117481.155395,94079.210957,50405.638583,20265.970739,23407.601636,4.127815,0.065669
2,0.3,3514,660,294,1575,0.842131,117481.155395,89100.876282,42146.128781,25410.249903,21544.497598,3.853080,0.061298
3,0.4,3762,412,379,1490,0.869105,117481.155395,84292.257562,35866.638453,30379.155913,18046.463196,3.272251,0.052058
4,0.5,3950,224,448,1421,0.888797,117481.155395,80388.790602,31020.305076,34412.738440,14955.747086,2.742714,0.043634
5,0.6,4085,89,542,1327,0.895582,117481.155395,75071.024017,26701.976892,39907.763910,8461.283214,1.576157,0.025075
6,0.7,4142,32,639,1230,0.888962,117481.155395,69583.541478,23797.948332,45578.162534,207.430611,0.039279,0.000625
7,0.8,4171,3,767,1102,0.872580,117481.155395,62342.327405,20837.347787,53060.750409,-11555.770791,-2.236976,-0.035588
8,0.9,4174,0,1016,853,0.831872,117481.155395,48255.903155,16085.301052,67616.722135,-35446.120031,-7.172860,-0.114113


- What is the threshold with the highest accuracy, using strategy B?  
**0.6**  
- What is the threshold with the highest profit, using strategy B? What is the highest profit, using strategy B?  
**trashold = 0.2; total profit = 23407.60**  
- What is the threshold with the highest profit per customer, using strategy B ? What is the highest profit per customer, using strategy B?  
**trashold = 0.2; profit per customer = 4.13**  
- What is the ratio of profit per customer (obtained on the previous step) and p?  
**0.04**  
- Which strategy yields the highest profit (A or B)? What are the TP, FP, TN, FN in this case? What is the highest profit in that case?  
**strategy = B; tn, fp, fn, tp = (3164, 1010, 206, 1663); total profit = 23407.60**  
- Which strategy yields the highest profit per customer (A or B)? What are the TP, FP, TN, FN in this case? What is the highest profit per customer in that case?  
**strategy = B; tn, fp, fn, tp = (3164, 1010, 206, 1663); profit per customer = 4.13**  

### Report
Strategy B leads to a higher profit since more customers not churn and provide the money. Strategy B is better for all calculated metrics (for the same thresholds) and should definitely be chosen over strategy A.  
Threshold 0.6 gives the highest accuracy for both strategies A and B. In this case, profit for the strategy B is 5.6 times more than using the strategy A. Despite the fact, that the highest profit per customer / p is calculated with thresholds 0.3-0.5 (rounding to 2 digits), threshold 0.6 should be used in order to get better accuracy => more plausible result. Threshold of 0.2 gives the highest profit and profit per customer for the strategy B. For strategy A highest values gives the threshold of 0.1  
In my opinion, True Positive, False Negative, and False Positive are the most important metrics of the confusion matrix. True Positive and False Negative contain all information about users that are going to churn => decreasing of the profit. Special attention must be paid to False Negative value as it represents the number of users that will churn "unexpectedly" and should be minimized. False Positive value should also be minimized as it represents the amount of money that will be spent on discounts on users that were not going to churn and receive the discount. 

## Bonus
- For the bonus 10 points you need to redo all computations, but now instead of average p you should use customer’s MonthlyCharges. All the results for the bonus task must be summarized in an additional report. You must provide a comparison of the results (with the regular case when you use p).

In [16]:
def metrics_bonus(cm, acceptance, discount):
    tn, fp, fn, tp = cm.ravel()
    accuracy = (tp + tn) / (tp + fp + fn + tn)
    
    TP = df[df.TP == 1]
    FN = df[df.FN == 1]
    FP = df[df.FP == 1]
    TN = df[df.TN == 1]
    TP_sum = TP.MonthlyCharges.sum()
    FN_sum = FN.MonthlyCharges.sum()
    FP_sum = FP.MonthlyCharges.sum()
    
    losses_do_nothing = TP_sum + FN_sum
    total_gains = TP_sum * acceptance
    total_costs = (TP_sum + FP_sum) * discount
    total_losses = FN_sum + TP_sum * (1 - acceptance) * (1 - discount)
    total_profit = total_gains - total_costs - total_losses
    profit_per_customer = total_profit / (len(TP) * acceptance + len(FP) + len(TN))

    return pd.DataFrame({'tn': [tn],
                         'fp': [fp],
                         'fn': [fn],
                         'tp': [tp],
                         'accuracy': [accuracy],
                         'losses_do_nothing': [losses_do_nothing],
                         'total_gains': [total_gains],
                         'total_costs': [total_costs],
                         'total_losses': [total_losses],
                         'total_profit': [total_profit],
                         'profit_per_customer': [profit_per_customer],
                         'PPC/p': [profit_per_customer / p]})

In [17]:
strategy_A = pd.DataFrame(columns=['threshold', 'tn', 'fp', 'fn', 'tp', 'accuracy', 'losses_do_nothing', 'total_gains', 'total_costs', 'total_losses', 'total_profit', 'profit_per_customer'])
for i in range(1, 10):
    threshold = .1 * i
    
    lr = LogisticRegression(max_iter=1000)
    lr_proba = cross_val_predict(lr, X, y, method='predict_proba')
    lr_pred = [int(item[1] >= threshold) for item in lr_proba]
    
    df = tel.copy()
    df['PredictedChurn'] = lr_pred
    
    df['TN'] = (df.Churn == 0) & (df.PredictedChurn == 0)
    df['FP'] = (df.Churn == 0) & (df.PredictedChurn == 1)
    df['FN'] = (df.Churn == 1) & (df.PredictedChurn == 0)
    df['TP'] = (df.Churn == 1) & (df.PredictedChurn == 1)

    df['TN'] = df.TN.astype(int)
    df['FP'] = df.FP.astype(int)
    df['FN'] = df.FN.astype(int)
    df['TP'] = df.TP.astype(int)
    
    cm = confusion_matrix(y, lr_pred)
    tn, fp, fn, tp = cm.ravel()
    
    m = metrics_bonus(cm, 0.75, 0.2)
    m['threshold'] = threshold 
    strategy_A = strategy_A.append(m, sort=False, ignore_index=True)

strategy_A

,threshold,tn,fp,fn,tp,accuracy,losses_do_nothing,total_gains,total_costs,total_losses,total_profit,profit_per_customer,PPC/p
0,0.1,2653,1521,121,1748,0.728281,139130.85,98148.2625,45899.78,34439.37,17809.1125,3.246876,0.051654
1,0.2,3164,1010,206,1663,0.798775,139130.85,94460.5875,38573.07,38372.89,17514.6275,3.230736,0.051398
2,0.3,3514,660,294,1575,0.842131,139130.85,90509.5125,33259.35,42587.37,14662.7925,2.738022,0.043559
3,0.4,3762,412,379,1490,0.869105,139130.85,86458.1625,29022.78,46908.81,10526.5725,1.989336,0.031648
4,0.5,3950,224,448,1421,0.888797,139130.85,82914.1125,25412.28,50689.13,6812.7025,1.300196,0.020685
5,0.6,4085,89,542,1327,0.895582,139130.85,78021.1125,22154.30,55908.33,-41.5175,-0.008032,-0.000128
6,0.7,4142,32,639,1230,0.888962,139130.85,72901.2375,19945.69,61369.53,-8413.9825,-1.650933,-0.026265
7,0.8,4171,3,767,1102,0.872580,139130.85,66314.1375,17733.71,68395.77,-19815.3425,-3.962672,-0.063042
8,0.9,4174,0,1016,853,0.831872,139130.85,52310.7375,13949.53,83332.73,-44971.5225,-9.342305,-0.148626


In [18]:
strategy_B = pd.DataFrame(columns=['threshold', 'tn', 'fp', 'fn', 'tp', 'accuracy', 'losses_do_nothing', 'total_gains', 'total_costs', 'total_losses', 'total_profit', 'profit_per_customer'])
for i in range(1, 10):
    threshold = .1 * i
    
    lr = LogisticRegression(max_iter=1000)
    lr_proba = cross_val_predict(lr, X, y, method='predict_proba')
    lr_pred = [int(item[1] >= threshold) for item in lr_proba]
    
    df = tel.copy()
    df['PredictedChurn'] = lr_pred
    
    df['TN'] = (df.Churn == 0) & (df.PredictedChurn == 0)
    df['FP'] = (df.Churn == 0) & (df.PredictedChurn == 1)
    df['FN'] = (df.Churn == 1) & (df.PredictedChurn == 0)
    df['TP'] = (df.Churn == 1) & (df.PredictedChurn == 1)

    df['TN'] = df.TN.astype(int)
    df['FP'] = df.FP.astype(int)
    df['FN'] = df.FN.astype(int)
    df['TP'] = df.TP.astype(int)
    
    cm = confusion_matrix(y, lr_pred)
    tn, fp, fn, tp = cm.ravel()
    
    m = metrics_bonus(cm, 0.9, 0.3)
    m['threshold'] = threshold 
    strategy_B = strategy_B.append(m, sort=False, ignore_index=True)

strategy_B

,threshold,tn,fp,fn,tp,accuracy,losses_do_nothing,total_gains,total_costs,total_losses,total_profit,profit_per_customer,PPC/p
0,0.1,2653,1521,121,1748,0.728281,139130.85,117777.915,68849.670,17427.0045,31501.2405,5.481146,0.087199
1,0.2,3164,1010,206,1663,0.798775,139130.85,113352.705,57859.605,21999.7215,33493.3785,5.906392,0.093964
2,0.3,3514,660,294,1575,0.842131,139130.85,108611.415,49889.025,26899.0545,31823.3355,5.691377,0.090544
3,0.4,3762,412,379,1490,0.869105,139130.85,103749.795,43534.170,31922.7285,28292.8965,5.130172,0.081616
4,0.5,3950,224,448,1421,0.888797,139130.85,99496.935,38118.420,36317.3505,25061.1645,4.595933,0.073116
5,0.6,4085,89,542,1327,0.895582,139130.85,93625.335,33231.450,42384.6705,18009.2145,3.354733,0.053370
6,0.7,4142,32,639,1230,0.888962,139130.85,87481.485,29918.535,48733.3155,8829.6345,1.671963,0.026599
7,0.8,4171,3,767,1102,0.872580,139130.85,79576.965,26600.565,56901.3195,-3924.9195,-0.759789,-0.012087
8,0.9,4174,0,1016,853,0.831872,139130.85,62772.885,20924.295,74265.5355,-32416.9455,-6.559877,-0.104361


### Report
All statements from main report hold for these calculations too (except numbers).